In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
/kaggle/input/child-mind-institute-problematic-internet-use/train.csv
/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=00115b9f/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=001f3379/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0745c390/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=eaab7a96/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=8ec2cc63/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=b2987a65/part-0.parquet
/kaggle/input/child-mind-institute-problematic-intern

# Prétraitement des données


1. Chargement et combinaison des données 

In [2]:
#  les données de la competition proviennent de deux types de fichiers : parquet et csv donc avant de passer au pretraitement on doit les combiner 
from tqdm import tqdm
def process_file(file_path):
    df = pd.read_parquet(file_path)
    if 'step' in df.columns:
        df = df.drop('step', axis=1)
    stats = df.describe().values.reshape(-1)
    id_value = os.path.basename(os.path.dirname(file_path)).split('=')[1]
    return stats, id_value

def load_time_series(directory):
    ids = os.listdir(directory)
    all_stats = []
    all_ids = []
    for id_dir in tqdm(ids):
        file_path = os.path.join(directory, id_dir, 'part-0.parquet')
        if os.path.exists(file_path):
            stats, id_value = process_file(file_path)
            all_stats.append(stats)
            all_ids.append(id_value)
    num_features = len(all_stats[0])
    columns = [f"TS_Feature_{i}" for i in range(num_features)]
    df = pd.DataFrame({
        'id': all_ids
    })
    df[columns] = all_stats
    return df

series_train_df = load_time_series('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet')
series_test_df = load_time_series('/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet')

# Load the datasets
tabular_train_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
tabular_test_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

train_df = pd.merge(tabular_train_df, series_train_df, how='left', on='id')
test_df = pd.merge(tabular_test_df, series_test_df, how='left', on='id')


100%|██████████| 2/2 [00:00<00:00,  5.92it/s]


2. Traitement des valeurs manquantes des données d'entrainement:
   * on fait appel à la fonction select(num_features contient les noms des colonnes de type numérique , cat_features contient les noms des colonnes de types categorielle )

In [3]:
num_features = train_df.select_dtypes(include= ['int64' , 'float64']).columns
cat_features = train_df.select_dtypes(include = 'object').columns

* les noms des colonnes qui ont des valeurs manquantes des données d'entrainement

In [4]:
print(train_df.isnull().sum().sort_values(ascending=False) [:50])

PAQ_A-Season                    3485
PAQ_A-PAQ_A_Total               3485
Fitness_Endurance-Time_Sec      3220
Fitness_Endurance-Time_Mins     3220
Fitness_Endurance-Max_Stage     3217
Physical-Waist_Circumference    3062
TS_Feature_7                    2964
TS_Feature_34                   2964
TS_Feature_35                   2964
TS_Feature_36                   2964
TS_Feature_37                   2964
TS_Feature_38                   2964
TS_Feature_39                   2964
TS_Feature_40                   2964
TS_Feature_42                   2964
TS_Feature_41                   2964
TS_Feature_32                   2964
TS_Feature_43                   2964
TS_Feature_44                   2964
TS_Feature_45                   2964
TS_Feature_46                   2964
TS_Feature_47                   2964
TS_Feature_48                   2964
TS_Feature_49                   2964
TS_Feature_50                   2964
TS_Feature_33                   2964
TS_Feature_30                   2964
T

* Remplacer les valeurs manquantes numériques avec median

In [5]:
for i in num_features:
    train_df[i] = train_df[i].fillna(train_df[i].median())
    

* Remplacer les valeurs manquantes catégorielles avec la valeur la plus fréquente dans la colonne correspendante 

In [6]:
for col in cat_features:
    most_frequent_value = train_df[col].mode()[0]  
    train_df[col] = train_df[col].fillna(most_frequent_value)  

3.  Suppression des colonnes **`id`** , **`sii`** et création des cibles **`y_train`**. :

In [7]:
X_train = train_df.drop(['id', 'sii'], axis = 1)
Y_train = train_df['sii']
print(X_train.shape)
print(train_df.shape[1])
print("Valeurs de y_train (sii) :")
print(Y_train.head())
print(Y_train.unique())

(3960, 176)
178
Valeurs de y_train (sii) :
0    2.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: sii, dtype: float64
[2. 0. 1. 3.]


In [8]:
print(X_train.shape[1])

176


4. Encodage des variables catégorielles de l'entrainement:

In [9]:
X_train = pd.get_dummies(X_train)

* Aprés avoir encodé les valeurs categorielles , on aura plus de colonnes du coup le num_features n'est plus le meme , donc on redefinit num_features 

In [10]:
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns

5. Division des données d'entraînement  en ensembles d'entraînement et de validation:


In [11]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print("Colonnes de x_train :", x_train.shape)
print("Colonnes de x_val :", x_val.shape) 


Colonnes de x_train : (3168, 209)
Colonnes de x_val : (792, 209)


6. Normalisation des données numériques d'entrainement et de validation :

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train[num_features] = scaler.fit_transform(x_train[num_features])  # Normaliser x_train
x_val[num_features] = scaler.transform(x_val[num_features])  # Normaliser x_val avec le même scaler


7. Traitement des valeurs manquantes des données de test 

In [13]:
num_features_test = test_df.select_dtypes(include= ['int64' , 'float64']).columns
cat_features_test = test_df.select_dtypes(include = 'object').columns
print(test_df.columns)
print(test_df.shape[1])

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight',
       ...
       'TS_Feature_86', 'TS_Feature_87', 'TS_Feature_88', 'TS_Feature_89',
       'TS_Feature_90', 'TS_Feature_91', 'TS_Feature_92', 'TS_Feature_93',
       'TS_Feature_94', 'TS_Feature_95'],
      dtype='object', length=155)
155


* les noms des colonnes qui ont des valeurs manquantes des données de test

In [14]:
print(test_df.isnull().sum().sort_values(ascending=False) [:50])

PAQ_A-PAQ_A_Total    19
PAQ_A-Season         19
TS_Feature_18        18
TS_Feature_47        18
TS_Feature_42        18
TS_Feature_43        18
TS_Feature_44        18
TS_Feature_45        18
TS_Feature_46        18
TS_Feature_48        18
TS_Feature_40        18
TS_Feature_49        18
TS_Feature_50        18
TS_Feature_51        18
TS_Feature_52        18
TS_Feature_53        18
TS_Feature_54        18
TS_Feature_55        18
TS_Feature_41        18
TS_Feature_39        18
TS_Feature_20        18
TS_Feature_29        18
TS_Feature_22        18
TS_Feature_23        18
TS_Feature_24        18
TS_Feature_25        18
TS_Feature_26        18
TS_Feature_27        18
TS_Feature_28        18
TS_Feature_30        18
TS_Feature_38        18
TS_Feature_31        18
TS_Feature_32        18
TS_Feature_33        18
TS_Feature_34        18
TS_Feature_35        18
TS_Feature_36        18
TS_Feature_37        18
TS_Feature_56        18
TS_Feature_57        18
TS_Feature_58        18
TS_Feature_86   

* Remplacer les valeurs manquantes numériques des données de test avec median d'entrainement

In [15]:
for i in num_features_test:
    test_df[i] = train_df[i].fillna(train_df[i].median())

* Remplacer les valeurs manquantes catégorielles dans les données de test  avec la valeur la plus fréquente dans la colonne correspendante (des données d'entrainement toujours)

In [16]:
for col in cat_features_test:
    most_frequent_value = train_df[col].mode()[0]  # Trouver la valeur la plus fréquente
    test_df[col] = test_df[col].fillna(most_frequent_value)  # Remplacer les valeurs manquantes



* Suppression de la colonne **`id`** dans les données de test

In [17]:
Test_df = test_df.drop(['id'], axis = 1)

* Normalisation des données numérique de test 

In [18]:
 from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Test_df[num_features_test] = scaler.fit_transform(Test_df[num_features_test])  
print(Test_df.shape[1])

154


* Encodage des variables catégorielles dans les données de test

In [19]:
Test_df = pd.get_dummies(Test_df)

* Aprés avoir encodé les valeurs categorielles , on aura plus de colonnes du coup le num_features_test n'est plus le meme , donc on redefinit num_features 

In [20]:
num_features_test = Test_df.select_dtypes(include=['int64', 'float64']).columns 
print(Test_df.shape[1])

180


In [21]:
print("Nombre de colonnes dans X_train aprés l'encodage des données  :", X_train.shape[1])
print("Nombre de colonnes dans test_df aprés l'encodage de données  :", Test_df.shape[1])

Nombre de colonnes dans X_train aprés l'encodage des données  : 209
Nombre de colonnes dans test_df aprés l'encodage de données  : 180


8. Alignement des colonnes entre les ensembles d'entraînement et de test :

    * On remarque que l'ensemble d'entrainement a plus de colonnes que l'ensemble de test
    * On Ajoute les colonnes manquantes dans test_df avec des valeurs 0
    * On Réorganise les colonnes de test_df pour correspondre à l'ordre de X_train

In [22]:
columns_only_in_train = set(X_train.columns) - set(Test_df.columns)
print("Colonnes uniquement dans X_train :", columns_only_in_train)


for col in columns_only_in_train:
    Test_df[col] = 0


Test_df = Test_df[X_train.columns]
print("Nombre de colonnes dans X_train :", X_train.shape[1])
print("Nombre de colonnes dans Test_df :", Test_df.shape[1])

Colonnes uniquement dans X_train : {'PCIAT-PCIAT_07', 'PCIAT-PCIAT_05', 'PCIAT-Season_Spring', 'Fitness_Endurance-Season_Winter', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_20', 'PCIAT-PCIAT_13', 'PCIAT-Season_Summer', 'PCIAT-Season_Fall', 'PAQ_A-Season_Spring', 'PCIAT-PCIAT_Total', 'BIA-Season_Spring', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_15', 'PAQ_A-Season_Fall', 'PCIAT-Season_Winter', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_03'}
Nombre de colonnes dans X_train : 209
Nombre de colonnes dans Test_df : 209


In [23]:
import keras
import tensorflow as tf
from tensorflow import keras
# modele lineare 
model_linear = keras.models.Sequential([
    keras.layers.Dense(1, input_dim=x_train.shape[1], activation='linear')  # Modèle linéaire
])
keras.utils.set_random_seed(42)
model_linear.compile(loss='mean_squared_error', 
              optimizer='adam', 
              metrics=['mean_absolute_error'])

exp_name = "linear_regression_batch32_epochs100"
model_linear.fit(x_train, y_train, 32, 100,
                  callbacks=[keras.callbacks.TensorBoard(log_dir=f'./logs/{exp_name}')])
val_loss, val_mae = model_linear.evaluate(x_val, y_val)
print(f'Validation Loss: {val_loss}, Validation MAE: {val_mae}')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3819 - mean_absolute_error: 0.7859
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5816 - mean_absolute_error: 0.4808
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3604 - mean_absolute_error: 0.3814
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2476 - mean_absolute_error: 0.3243
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1837 - mean_absolute_error: 0.2905
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1457 - mean_absolute_error: 0.2691
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1228 - mean_absolute_error: 0.2546
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1090 - mean_absolute_error: 0.2453
Epoch 9/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1005 - mean_absolute_error: 0.2387
Epoch 10/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0952 - mean_absolute_error: 0.2336
Epoch 11/100
99/99 ━━━━━━━━━━━━━━━━━━━━

* Générer les prédictions 

In [24]:
predictions = model_linear.predict(Test_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


# Submission 

In [25]:
# Vérifiez la taille et le type de predictions
print("Taille de predictions avant transformation :", predictions.shape)
print("Type de predictions avant transformation :", type(predictions))

# Convertir predictions en un tableau unidimensionnel
predictions = predictions.ravel()  # Cela transforme (20, 1) en (20,)

# Convertir les prédictions en entiers (si c'est un problème de classification avec des classes discrètes)
predictions = predictions.round().astype(int)  # Arrondir et convertir en entiers

# Créez un DataFrame pour les soumissions
submission_df = pd.DataFrame({
    'id': test_df['id'].values,  # Utilisez .values pour obtenir un tableau numpy
    'sii': predictions  # predictions est maintenant unidimensionnel et entier
})

# Assurez-vous que 'id' est de type chaîne de caractères et remplissez avec des zéros pour avoir 8 caractères
submission_df['id'] = submission_df['id'].astype(str).str.zfill(8)
# Vérifiez le contenu et les types de données dans submission_df
print(submission_df.head())
print(submission_df.dtypes)

# Sauvegardez le DataFrame dans un fichier CSV
submission_df.to_csv('submission.csv', index=False)

print("Submission DataFrame créé avec succès.")

Taille de predictions avant transformation : (20, 1)
Type de predictions avant transformation : <class 'numpy.ndarray'>
         id  sii
0  00008ff9    0
1  000fd460    0
2  00105258    0
3  00115b9f    0
4  0016bb22    0
id     object
sii     int64
dtype: object
Submission DataFrame créé avec succès.


In [26]:
submission_generated = pd.read_csv('/kaggle/working/submission.csv')
submission_example = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')
# Vérifier le nombre de lignes et de colonnes
if submission_generated.shape == submission_example.shape:
    print("Les fichiers ont le même nombre de lignes et de colonnes.")
else:
    print("Les fichiers n'ont pas le même nombre de lignes ou de colonnes.")

# Vérifier les types de colonnes
if submission_generated.dtypes.equals(submission_example.dtypes):
    print("Les types de colonnes correspondent.")
else:
    print("Les types de colonnes ne correspondent pas.")
    print("Types dans le fichier généré:", submission_generated.dtypes)
    print("Types dans l'exemple:", submission_example.dtypes)

comparison = submission_generated.head(10) == submission_example.head(10)
if comparison.all().all():
    print("Les 10 premières lignes sont identiques.")
else:
    print("Les 10 premières lignes diffèrent.")
    print(submission_generated.head(10))
    print(submission_example.head(10))

# 4. Vérification de l'égalité exacte des identifiants
ids_identiques = submission_generated['id'].equals(submission_example['id'])
print("\nLes identifiants sont-ils identiques dans les deux fichiers ? :", ids_identiques)



Les fichiers ont le même nombre de lignes et de colonnes.
Les types de colonnes correspondent.
Les 10 premières lignes diffèrent.
         id  sii
0  00008ff9    0
1  000fd460    0
2  00105258    0
3  00115b9f    0
4  0016bb22    0
5  001f3379    1
6  0038ba98    0
7  0068a485    0
8  0069fbed    0
9  0083e397    0
         id  sii
0  00008ff9    0
1  000fd460    1
2  00105258    2
3  00115b9f    3
4  0016bb22    0
5  001f3379    1
6  0038ba98    2
7  0068a485    3
8  0069fbed    0
9  0083e397    1

Les identifiants sont-ils identiques dans les deux fichiers ? : True
